### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [51]:
from collections import namedtuple, deque
from itertools import tee, islice
from blist import sortedlist
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000): #10000000
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [2]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(x) for x in result / count]

In [3]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print(get_tuple_stream_mean(example(medium_stream()), 2))

['127.48', '137.48']
Wall time: 2min 1s


In [6]:
def get_stream_mean_and_variance(stream, window_size):
    start, end = tee(stream)
    sum_1 = 0.
    sum_2 = 0.
    for _ in range(window_size):
        value = next(end)
        sum_1 += value
        sum_2 += value * value
    try:
        while True:
            yield sum_1 / window_size, (sum_2 - sum_1 * sum_1 / window_size) / window_size

            next_val = next(end)
            prev_val = next(start)
            
            sum_1 += (next_val - prev_val)
            sum_2 += (next_val * next_val - prev_val * prev_val)
    except StopIteration:
        pass

In [35]:
%%time
get_tuple_stream_mean(get_stream_mean_and_variance(easy_stream(), 1000), 2)

Wall time: 3min 29s


['4999675.28', '83438.25']

In [5]:
%%time
get_tuple_stream_mean(get_stream_mean_and_variance(medium_stream(), 1000), 2)

Wall time: 2min 30s


['127.48', '5455.17']

In [7]:
%%time
get_tuple_stream_mean(get_stream_mean_and_variance(nightmare_stream(), 1000), 2)

Wall time: 2min 38s


['499880345.88', '83228908564114592.00']

In [17]:
def get_easy_stream_min_median_max(stream, window_size):
    queue = deque()
    for value in islice(stream, window_size):
        queue.append(value)
    
    yield queue[0], queue[int(window_size / 2)], queue[-1]
    for value in stream:
        queue.popleft()
        queue.append(value)
        yield queue[0], queue[int(window_size / 2)], queue[-1]

In [18]:
%%time
get_tuple_stream_mean(get_easy_stream_min_median_max(easy_stream(), 1000), 3)

Wall time: 2min 41s


['4999175.79', '4999675.78', '5000174.76']

In [43]:
def get_easy_stream_min_median_max_iter(stream, window_size):
    start, medium, end = tee(stream, 3)
    for _ in range(int(window_size/2)):
        next(medium)
    for _ in range(int(window_size-1)):
        next(end)
    yield next(start), next(medium), next(end)
    try:
        while True:
            yield next(start), next(medium), next(end)
    except StopIteration:
        pass

In [44]:
%%time
get_tuple_stream_mean(get_easy_stream_min_median_max_iter(easy_stream(), 1000), 3)

Wall time: 3min 2s


['4999175.79', '4999675.78', '5000174.76']

In [60]:
def get_stream_min_median_max(stream, window_size):
    window = sortedlist()
    start, end = tee(stream)
    for _ in range(window_size):
        window.add(next(end))
    try:
        while True:
            yield window[0], window[int(window_size / 2)], window[-1]
            window.add(next(end))
            window.remove(next(start))
    except StopIteration:
        pass

In [61]:
%%time
get_tuple_stream_mean(get_stream_min_median_max(medium_stream(), 1000), 3)

Wall time: 5min 27s


['0.02', '127.60', '254.98']

In [62]:
%%time
get_tuple_stream_mean(get_stream_min_median_max(nightmare_stream(), 1000), 3)

Wall time: 6min 59s


['1017512.29', '500438415.64', '999017359.97']